In [1]:
import os
import glob
import json
import csv
import json
import pandas as pd
from tqdm import tqdm
import zstandard as zstd
# from bertopic import BERTopic
import matplotlib.pyplot as plt

In [2]:
# Directory containing .zst files
source_dir = 'subreddits'
# Directory to store extracted files
dest_dir = 'subs_extracted'

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# Get list of all .zst files
zst_files = glob.glob(os.path.join(source_dir, '*.zst'))

# Get list of already extracted files (without the .zst extension)
extracted_files = {os.path.splitext(os.path.basename(f))[0] for f in os.listdir(dest_dir)}

# Filter out files that have already been extracted
files_to_extract = [f for f in zst_files if os.path.splitext(os.path.basename(f))[0] not in extracted_files]

# Initialize tqdm progress bar
with tqdm(total=len(files_to_extract), desc="Extracting files", unit="file") as pbar:
    for zst_file in files_to_extract:
        with open(zst_file, 'rb') as compressed:
            dctx = zstd.ZstdDecompressor()
            with dctx.stream_reader(compressed) as reader:
                contents = reader.read()

            # Create a corresponding file name for the decompressed data
            base_name = os.path.basename(zst_file)
            decompressed_file_name = os.path.join(dest_dir, base_name.replace('.zst', ''))

            # Write the decompressed data to a new file in the destination directory
            with open(decompressed_file_name, 'wb') as decompressed_file:
                decompressed_file.write(contents)

        # Update the progress bar
        pbar.update(1)


Extracting files: 100%|███████████████████████| 24/24 [00:40<00:00,  1.69s/file]


In [6]:
from datetime import datetime
# Define the date range
start_date = datetime(2017, 12, 1).timestamp()
end_date = datetime(2022, 11, 30).timestamp()

# Function to convert Unix timestamp to readable format
def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# Load submissions and comments from files
with open('subs_extracted/StudentLoans_submissions') as file:
    submissions = [json.loads(line) for line in file]

with open('subs_extracted/StudentLoans_comments') as file:
    comments = [json.loads(line) for line in file]

# Filter submissions within the date range and having text
submissions = [
    sub for sub in submissions
    if 'created_utc' in sub and
    start_date <= float(sub['created_utc']) <= end_date and
    'selftext' in sub and
    sub['selftext'].strip()
]


# Create a mapping from submission ID to comments
comments_by_submission = {}
for comment in comments:
    if 'body' in comment and comment['body'].strip():
        submission_id = comment['link_id'][3:]  # Remove 't3_' prefix
        comments_by_submission.setdefault(submission_id, []).append(comment['body'])

# Write to CSV
with open('StudentLoans.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['submission_id', 'num_crossposts', 'gilded', 'can_gild', 'submission_text', 'contest_mode', 'brand_safe', 'score', 'submission_title', 'submission_comments', 'submission_created_date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for sub in submissions:
        writer.writerow({
            'submission_id': sub['id'],
            'num_crossposts': sub.get('num_crossposts', 0),
            'gilded': sub.get('gilded', 0),
            'can_gild': sub.get('can_gild', False),
            'submission_text': sub['selftext'],
            'contest_mode': sub.get('contest_mode', False),
            'brand_safe': sub.get('brand_safe', True),
            'score': sub.get('score', 0),
            'submission_title': sub.get('title', ''),
            'submission_comments': ' | '.join(comments_by_submission.get(sub['id'], [])),
            'submission_created_date': convert_timestamp(sub['created_utc'])
        })


In [3]:
from datetime import datetime
import json
import csv

# Prompt for file paths
submissions_file = input("Enter the path to the submissions file: ")
comments_file = input("Enter the path to the comments file: ")
output_csv_file = input("Enter the path for the output CSV file: ")

# Prompt for start and end dates
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")

# Convert string dates to timestamps
start_date = datetime.strptime(start_date_str, '%Y-%m-%d').timestamp()
end_date = datetime.strptime(end_date_str, '%Y-%m-%d').timestamp()

# Function to convert Unix timestamp to readable format
def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# Load submissions and comments from files
with open(submissions_file) as file:
    submissions = [json.loads(line) for line in file]

with open(comments_file) as file:
    comments = [json.loads(line) for line in file]

# Filter submissions within the date range and having text
submissions = [
    sub for sub in submissions
    if 'created_utc' in sub and
    start_date <= float(sub['created_utc']) <= end_date and
    'selftext' in sub and
    sub['selftext'].strip()
]

# Create a mapping from submission ID to comments
comments_by_submission = {}
for comment in comments:
    if 'body' in comment and comment['body'].strip():
        submission_id = comment['link_id'][3:]  # Remove 't3_' prefix
        comments_by_submission.setdefault(submission_id, []).append(comment['body'])

# Write to CSV
with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['submission_id', 'num_crossposts', 'gilded', 'can_gild', 'submission_text', 'contest_mode', 'brand_safe', 'score', 'submission_title', 'submission_comments', 'submission_created_date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for sub in submissions:
        writer.writerow({
            'submission_id': sub['id'],
            'num_crossposts': sub.get('num_crossposts', 0),
            'gilded': sub.get('gilded', 0),
            'can_gild': sub.get('can_gild', False),
            'submission_text': sub['selftext'],
            'contest_mode': sub.get('contest_mode', False),
            'brand_safe': sub.get('brand_safe', True),
            'score': sub.get('score', 0),
            'submission_title': sub.get('title', ''),
            'submission_comments': ' | '.join(comments_by_submission.get(sub['id'], [])),
            'submission_created_date': convert_timestamp(sub['created_utc'])
        })


Enter the path to the submissions file: subs_extracted/boston_submissions
Enter the path to the comments file: subs_extracted/boston_comments
Enter the path for the output CSV file: subs_csv_data/boston.csv
Enter the start date (YYYY-MM-DD): 2019-12-01
Enter the end date (YYYY-MM-DD): 2022-12-31


In [4]:
reddit = pd.read_csv('subs_csv_data/boston.csv')
print(reddit.shape)
reddit.head()

(47879, 11)


,submission_id,num_crossposts,gilded,can_gild,submission_text,contest_mode,brand_safe,score,submission_title,submission_comments,submission_created_date
0,e4b472,0,0,True,"New grad from NY, just moved here for a job of...",False,True,4,Just moved here and very nervous,Sounds like maybe you should get a t pass | Us...,2019-12-01 05:11:20
1,e4facd,0,0,True,[removed],False,True,0,WHAT AAL REFERS TO,American Alcohol League. It’s the drinking org...,2019-12-01 10:03:39
2,e4hi1g,0,0,False,They are asking people to do that because flig...,False,True,0,Check your flights and call ahead before headi...,Thank goodness you are looking out for people....,2019-12-01 13:34:52
3,e4i294,0,0,True,"I’ve never taken the green line when it snows,...",False,True,0,How bad is the green line going to be tomorrow...,[deleted] | It won't be anything out of the or...,2019-12-01 14:26:48
4,e4i2fw,0,0,True,First winter where I’ll keep my car in the cit...,False,True,0,Threshold for Boston snow emergency/parking ban?,"Not anymore. A few years ago, there was a park...",2019-12-01 14:27:16
